In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report
import seaborn as sns
from scipy import stats
from pathlib import Path
import pickle
import numpy as np

In [2]:
# display all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 70)

In [3]:
cd tmp

/home/team9/tmp


In [4]:
ls

data/


In [5]:
cd data

/home/team9/tmp/data


In [6]:
ls

Custom_location_53_345035_-6_267261_62b5c8e6c91d98000ba01ceb.csv
leavetimes/
routes/
routes_no_outliers/
rt_leavetimes_DB_2018.txt
rt_trips_DB_2018.txt
rt_vehicles_DB_2018.txt
rt_vehicles_DB_2018.zip
test/
test_46a_dask.csv
trips_modelling.csv
trips_ready.csv
weather_factorised.csv
weather_ready.csv
weather_trips.csv


## get unique tripid with lineid with direction

In [7]:
trips = pd.read_csv('rt_trips_DB_2018.txt', sep=";", parse_dates=[0],usecols=['DAYOFSERVICE', 'TRIPID', 'LINEID'])

In [8]:
trips

,DAYOFSERVICE,TRIPID,LINEID
0,2018-02-07,6253783,68
1,2018-02-07,6262138,25B
2,2018-02-07,6254942,45A
3,2018-02-07,6259460,25A
4,2018-02-07,6253175,14
...,...,...,...
2182632,2018-05-14,6765849,123
2182633,2018-05-14,6765469,75
2182634,2018-05-14,6765486,33D
2182635,2018-05-14,6764987,70


In [9]:
trips.nunique()

DAYOFSERVICE       360
TRIPID          658964
LINEID             130
dtype: int64

In [10]:
#trips = trips.drop_duplicates(subset=['TRIPID','LINEID','DIRECTION'],keep='first')
trips = trips.drop_duplicates(subset=['TRIPID','LINEID'],keep='first')

In [11]:
trips

,DAYOFSERVICE,TRIPID,LINEID
0,2018-02-07,6253783,68
1,2018-02-07,6262138,25B
2,2018-02-07,6254942,45A
3,2018-02-07,6259460,25A
4,2018-02-07,6253175,14
...,...,...,...
2178053,2018-05-27,6841018,11
2178054,2018-05-27,6841376,76
2178055,2018-05-27,6846574,76
2179536,2018-05-15,6762495,25A


In [12]:
trips.nunique()

DAYOFSERVICE       347
TRIPID          658964
LINEID             130
dtype: int64

In [13]:
#trips = trips[['TRIPID','LINEID','DIRECTION']]
trips = trips[['TRIPID','LINEID']]

In [14]:
trips.sort_values('TRIPID')

,TRIPID,LINEID
681660,5955221,14
723197,5955222,14
684398,5955223,14
719581,5955224,14
674864,5955225,14
...,...,...
26165,8592203,67
32940,8592204,67
57440,8592205,67
28895,8592206,67


## Read leavetimes

#### notice: the whole file is too large and always cause memory error, I ll just test clean it use chunksize(check details in 3.1stopchunk), and just import the needed columns

In [15]:
leavetimes = pd.read_csv('rt_leavetimes_DB_2018.txt', sep=";",parse_dates=[0], usecols=['DAYOFSERVICE',
 'TRIPID',
 'PROGRNUMBER',
 'STOPPOINTID',
 'PLANNEDTIME_ARR',
 'PLANNEDTIME_DEP',
 'ACTUALTIME_ARR',
 'ACTUALTIME_DEP',
 'VEHICLEID'])

In [16]:
leavetimes.head(5)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID
0,2018-01-01,5972116,12,119,48030,48030,48012,48012,2693211
1,2018-01-01,5966674,12,119,54001,54001,54023,54023,2693267
2,2018-01-01,5959105,12,119,60001,60001,59955,59955,2693263
3,2018-01-01,5966888,12,119,58801,58801,58771,58771,2693284
4,2018-01-01,5965960,12,119,56401,56401,56309,56323,2693209


## Merge the tripid

In [17]:
leavetimes = pd.merge(leavetimes,trips,how='left')

In [18]:
#leavetimes['LINE_DIRECTION'] = leavetimes['LINEID'] + '_' + leavetimes['DIRECTION'].astype(str)

In [19]:
#leavetimes.nunique()

#### check if there's tripid without any corresponding lineid and direction

## divide by lineid

In [20]:
cd ~/tmp/data/test

/home/team9/tmp/data/test


In [21]:
line=leavetimes.LINEID.unique()
print(line)

['1' '13' '37' '104' '11' '102' '151' '41' '65' '7A' '140' '84' '79A'
 '15A' '17' '44' '40B' '41C' '83A' '238' '25' '38A' '63' '4' '66A' '31'
 '150' '17A' '25B' '31A' '33' '27B' '32' '14' '67' '9' '61' '43' '130'
 '68' '15' '123' '122' '120' '47' '69' '39A' '76' '40' '70' '29A' '25A'
 '38' '49' '65B' '39' '45A' '42' '77A' '27' '54A' '46A' '27A' '40D' '7'
 '53' '33B' '75' '270' '18' '16' '41B' '145' '83' '79' '16C' '66' '15B'
 '185' '26' '56A' '33A' '14C' '59' '220' '184' '142' '114' '76A' '31B'
 '68A' '66B' '111' '236' '84A' '67X' '84X' '239' '42D' '66X' '116' '41X'
 '46E' '38B' '32X' '161' '7B' '118' '25X' '38D' '7D' '44B' '33X' '25D'
 '51D' '15D' '31D' '39X' '27X' '51X' '77X' '69X' '41A' '70D' '68X' '33D'
 '40E' '16D' '33E' '41D']


Line:  
['1' '13' '37' '104' '11' '102' '151' '41' '65' '7A' '140' '84' '79A'
 '15A' '17' '44' '40B' '41C' '83A' '238' '25' '38A' '63' '4' '66A' '31'
 '150' '17A' '25B' '31A' '33' '27B' '32' '14' '67' '9' '61' '43' '130'
 '68' '15' '123' '122' '120' '47' '69' '39A' '76' '40' '70' '29A' '25A'
 '38' '49' '65B' '39' '45A' '42' '77A' '27' '54A' '46A' '27A' '40D' '7'
 '53' '33B' '75' '270' '18' '16' '41B' '145' '83' '79' '16C' '66' '15B'
 '185' '26' '56A' '33A' '14C' '59' '220' '184' '142' '114' '76A' '31B'
 '68A' '66B' '111' '236' '84A' '67X' '84X' '239' '42D' '66X' '116' '41X'
 '46E' '38B' '32X' '161' '7B' '118' '25X' '38D' '7D' '44B' '33X' '25D'
 '51D' '15D' '31D' '39X' '27X' '51X' '77X' '69X' '41A' '70D' '68X' '33D'
 '40E' '16D' '33E' '41D']

In [22]:
# line46A_df_list = []
# chunksize = 10 ** 6
# for chunk in pd.read_csv("rt_leavetimes_DB_2018.txt", sep=';', chunksize=chunksize):
#    line39A_df_list.append(chunk[chunk['TRIPID'].isin(tripid_39A_list)])

# df_line39A = pd.concat(line46A_df_list)

In [23]:
for i in line:
    df=leavetimes[leavetimes.LINEID ==i]
    df.to_csv('leavetimes_'+ i+'.csv',index=True)